In [ ]:
import pandas
import itertools
import csv

#creating a list of chromosomes to iterate through
chroms = ['chr%s' % i for i in range(1, 23) + ['X', 'Y', 'M']]

#create a list of pass subcategories
pass_subcats = ['PASS[0/1]', 'PASS[0]', 'PASS[.]', 'PASS[1]', 'PASS[1/1]', 'PASS[1/2]', 'PASS[0/0]']

#create a list of columns for the dataframe matrix. This represents all of the possible rcf filter
#categories
rcf_cats = ['not present', 'nc'] + pass_subcats

#create a list of indexes for the dataframe matrix. This represents all of the possible gvcf
#filter categories
gvcf_cats = ['not present']
with open('/40TB_3/InovaGenomes/Filter_combinations') as filters:
    filter_reader = csv.reader(filters, delimiter = '\t')
    for row in filter_reader:
        gvcf_cats.append(row[0])
gvcf_cats = gvcf_cats + pass_subcats
gvcf_cats.remove('PASS')

In [ ]:

#If there are two consecutive lines at the same position,
#will add that to the count. 

# when there is a deletion in the GVCF file, there will be at least 1 line and probably 2 lines
# in the RCF file. Also, the deletion does not get added to the matrix of counts. 

for chrom in chroms[24:25]: #testing chrM right now
    matrix = pandas.DataFrame(0, index= gvcf_cats, columns=rcf_cats)
    gvcf_stream_file = '%s.gvcf'%chrom
    gvcf_filepath = os.path.join('/40TB_3/gestalt/private/IndividualGenomeAssembly/LP6005636-DNA_F04/analyses/PosCat/',gvcf_stream_file) 
    rcf_stream_file = '%s.rcf'%chrom
    rcf_filepath = os.path.join('/40TB_3/gestalt/private/IndividualGenomeAssembly/LP6005636-DNA_F04/analyses/PosCat/', rcf_stream_file)
    with open(gvcf_filepath) as gvcf, open(rcf_filepath) as rcf:
        gvcf_reader = csv.reader(gvcf, delimiter = '\t')
        rcf_reader = csv.reader(rcf, delimiter = '\t')
        pass_00_row = next(gvcf_reader)
        pass_00_count = pass_00_row[0]
        matrix['not present'].loc['PASS[0/0]'] = pass_00_count
        last_rcf_row = ''
        last_gvcf_row = [0,'']
        flag = 0
        row_rcf = next(rcf_reader)
        for row_gvcf in gvcf_reader:
            while True:                
                rcf_pos = int(row_rcf[0])+1 # convert base 0 to base 1 to match gvcf
                gvcf_pos = int(row_gvcf[0])
                if gvcf_pos > 500:
                    flag  =1
                if gvcf_pos > 600 and flag ==1:
                    flag =0
                gvcf_cat = row_gvcf[1]
                rcf_cat = row_rcf[1]
                #last_rcf_row = row_rcf
                if flag:
                    print('gvcf {} rcf {} last_rcf {} last_gvcf {}'.format(gvcf_pos, rcf_pos, last_rcf_row, last_gvcf_row)) 
                if gvcf_pos == int(last_gvcf_row[0]):
                        if flag:
                            print("current gvcf  {} is equal to last gvcf {} row{}".format(gvcf_pos, last_gvcf_row[0], last_gvcf_row))
                        gvcf_pos += 1
          
                if (gvcf_pos == rcf_pos):
                    matrix.set_value(gvcf_cat, rcf_cat, matrix[rcf_cat].loc[gvcf_cat] + 1)
                    if(gvcf_cat == 'PASS[1/1]'):
                        print(gvcf_cat, gvcf_pos, rcf_cat,(rcf_pos), matrix[rcf_cat].loc[gvcf_cat])
                    try:
                        row_rcf = next(rcf_reader)
                    except:
                        break
                if gvcf_pos < rcf_pos:
                    if flag:
                        print("gvf pos less than rcf pos")   
                        print("last gvcf if less than {}".format(last_gvcf_row))
                        print(gvcf_cat, gvcf_pos, rcf_cat,rcf_pos, matrix[rcf_cat].loc[gvcf_cat])
                    break  
                if gvcf_pos > rcf_pos:
                    if flag:
                        print("gvcf > rcf {}".format(last_rcf_row))
                    break
            last_gvcf_row = row_gvcf

            
        #matrix.to_csv(path_or_buf ='category matrix for %s.csv'%chrom, sep = '\t', header = True, index = True)
        #matrix